In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import  VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
# Paramètres
input_shape = (224, 224, 3)
num_classes = 30  # Remplacez par le nombre de classes de votre tâche
batch_size = 32
epochs = 50

In [3]:
data_dir = "Segmented Medicinal Leaf Images"

In [4]:
#augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [5]:
# Chargez les données d'entraînement et de validation en utilisant le générateur
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size = input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Utilisez la sous-partie d'entraînement
)


Found 1479 images belonging to 30 classes.


In [6]:
test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Utilisez la sous-partie de validation
)

Found 356 images belonging to 30 classes.


In [7]:
# Chargement du modèle pré-entraîné VGG16 avec les poids ImageNet
base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)


80134624/80134624 [==============================] - 80s 1us/step


In [8]:
# Geler les couches du modèle de base (sauf les dernières couches de classification)
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [14]:
# Ajouter des couches personnalisées
x1 = base_model.output
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(1024, activation='relu')(x1)
predictions = Dense(num_classes, activation='softmax')(x1)

In [15]:
# Création du modèle final
model = Model(inputs=base_model.input, outputs=predictions)

In [16]:
# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Utilisation de l'arrêt précoce
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


In [18]:
# Entraînement du modèle
model.fit(
    train_generator,
    epochs=epochs,
    callbacks=[early_stopping],
    validation_data=test_generator  # Remplacez par votre générateur de validation
)

Epoch 1/50
47/47 [==============================] - 937s 20s/step - loss: 2.5395 - accuracy: 0.2732 - val_loss: 1.5938 - val_accuracy: 0.4888
Epoch 2/50
47/47 [==============================] - 925s 20s/step - loss: 0.9866 - accuracy: 0.6930 - val_loss: 0.9441 - val_accuracy: 0.6713
Epoch 3/50
47/47 [==============================] - 940s 20s/step - loss: 0.5649 - accuracy: 0.8087 - val_loss: 0.6545 - val_accuracy: 0.7809
Epoch 4/50
47/47 [==============================] - 940s 20s/step - loss: 0.5087 - accuracy: 0.8425 - val_loss: 0.5736 - val_accuracy: 0.8062
Epoch 5/50
47/47 [==============================] - 932s 20s/step - loss: 0.3595 - accuracy: 0.8844 - val_loss: 0.4862 - val_accuracy: 0.8539
Epoch 6/50
47/47 [==============================] - 937s 20s/step - loss: 0.2830 - accuracy: 0.9060 - val_loss: 0.4460 - val_accuracy: 0.8511
Epoch 7/50
47/47 [==============================] - 928s 20s/step - loss: 0.2711 - accuracy: 0.9128 - val_loss: 0.4295 - val_accuracy: 0.8596
Epoch 

In [20]:
# Évaluer les modèles individuels sur l'ensemble de test
test_loss1, test_accuracy1 = model.evaluate(test_generator)


print(" Accuracy:", test_accuracy1)

12/12 [==============================] - 162s 13s/step - loss: 0.1819 - accuracy: 0.9354
MobileNetV2 Accuracy: 0.9353932738304138


In [21]:
# Entraînement du modèle
model.fit(
    train_generator,
    epochs=10,
    callbacks=[early_stopping],
    validation_data=test_generator  # Remplacez par votre générateur de validation
)

Epoch 1/10
47/47 [==============================] - 967s 21s/step - loss: 0.0508 - accuracy: 0.9804 - val_loss: 0.3949 - val_accuracy: 0.8904
Epoch 2/10
47/47 [==============================] - 937s 20s/step - loss: 0.0546 - accuracy: 0.9817 - val_loss: 0.2166 - val_accuracy: 0.9270
Epoch 3/10
47/47 [==============================] - 929s 20s/step - loss: 0.0752 - accuracy: 0.9750 - val_loss: 0.2023 - val_accuracy: 0.9298
Epoch 4/10
47/47 [==============================] - 943s 20s/step - loss: 0.0735 - accuracy: 0.9784 - val_loss: 0.2038 - val_accuracy: 0.9326
Epoch 5/10
47/47 [==============================] - 935s 20s/step - loss: 0.0498 - accuracy: 0.9844 - val_loss: 0.2518 - val_accuracy: 0.9213
Epoch 6/10
47/47 [==============================] - 928s 20s/step - loss: 0.0466 - accuracy: 0.9831 - val_loss: 0.2642 - val_accuracy: 0.9242
Epoch 7/10
47/47 [==============================] - 933s 20s/step - loss: 0.0597 - accuracy: 0.9790 - val_loss: 0.6808 - val_accuracy: 0.8427
Epoch 

In [22]:
# Évaluer les modèles individuels sur l'ensemble de test
test_loss1, test_accuracy1 = model.evaluate(test_generator)


print("MobileNetV2 Accuracy:", test_accuracy1)

12/12 [==============================] - 159s 13s/step - loss: 0.2868 - accuracy: 0.9270
MobileNetV2 Accuracy: 0.9269663095474243
